In [4]:
import os
import yaml
import pandas as pd

In [5]:
# 2. Use the sample exploration config file to define a YAML configuration (config) file
path_to_yaml = os.path.join(os.getcwd(),'data_exploration_config.yml')
try:
    with open (path_to_yaml, 'r') as c_file:
        config = yaml.safe_load(c_file)
except Exception as e:
    print('Error reading the config file')
 

In [6]:
# 3. Add the code to your Jupyter notebook to ingest the config file you defined in the previous step 
# and create Python variables for the parameters you defined in the config file
load_from_scratch = config['general']['load_from_scratch']
save_raw_dataframe = config['general']['save_raw_dataframe']
save_transformed_dataframe = config['general']['save_transformed_dataframe']
remove_bad_values = config['general']['remove_bad_values']
categorical = config['columns']['categorical']
continuous = config['columns']['continuous']
date = config['columns']['date']
text = config['columns']['text']
excluded = config['columns']['excluded']
max_lat = config['bounding_box']['max_lat']
min_long = config['bounding_box']['min_long']
min_lat = config['bounding_box']['min_lat']
max_lat = config['newark_bounding_box']['max_lat']
min_long = config['newark_bounding_box']['min_long']
min_lat = config['newark_bounding_box']['min_lat']
geo_columns = config['geo_columns']
input_csv = config['file_names']['input_csv']
pickled_input_dataframe = config['file_names']['pickled_input_dataframe']
pickled_output_dataframe = config['file_names']['pickled_output_dataframe']

In [8]:
# 1. Load the Airbnb dataset into a pandas DataFrame
df = pd.read_pickle(pickled_input_dataframe)

In [12]:
#4. For each column category (categorical, continuous, and text), 
# determine an appropriate replacement value to replace missing values
print('Categorical columns:',str(categorical))
print('Continuous columns:',str(continuous+excluded))
print('Text columns:',str(text))
count_missing = (len(df) - df.count())
print(count_missing[count_missing != 0])

# For the name column we could use 'Anonymous' to fill in missing names and host names.
# for reviews_per_month column, we could fill missing values with 0, assuming no reviews.
# For missing last_review dates, if there are no reviews_per_month then, we could
# fill missing values with "January 1, 1970", as a specific date that clearly indicates 
# the absence of a review.

Categorical columns: ['neighbourhood_group', 'neighbourhood', 'room_type']
Continuous columns: ['minimum_nights', 'number_of_reviews', 'reviews_per_month', 'calculated_host_listings_count', 'latitude', 'longitude', 'price', 'id']
Text columns: ['name', 'host_name']
name                    16
host_name               21
last_review          10052
reviews_per_month    10052
dtype: int64


# Think about the pros and cons of a fixed value versus a calculated value

**Filling Missing Values with the Most Common Value (Fixed Value) for categorical**:
   - **Pros**: 
     - Ease of implementation: It's straightforward to implement and doesn't require complex computations.
     - Can work well for large datasets with minimal impact on overall accuracy.
   - **Cons**:
     - It may reduce accuracy, especially when dealing with smaller datasets. By imputing the most common value, you may lose some of the nuances in the data.
     - It doesn't consider the relationship between the missing value and other features, potentially leading to biased results.

   - **When to Use**: This approach is suitable for larger datasets where the impact on accuracy is minimal, or when the categorical feature isn't a critical determinant of the outcome.

2. **Filling Missing Values with Random Sampling Using Probability Density Function (PDF)**:
   - **Pros**:
     - Preserves the shape of the probability distribution: This method can provide a more nuanced representation of the data and maintain the distribution's characteristics.
     - Can be suitable for smaller datasets where preserving data distribution is crucial.

   - **Cons**:
     - Requires computations: Implementing random sampling based on PDF might be more computationally intensive than using the most common value.
     - Introduces noise: The randomness introduced by this method can add noise to the data, which might be undesirable in some cases.

   - **When to Use**: Random sampling based on PDF is more suitable when you have a smaller dataset and preserving the underlying data distribution is essential. However, you should be cautious about the potential introduction of noise, especially in larger datasets where data drift can occur.

In general, the choice between these two methods should also consider the specific context of your data and the impact of imputation on downstream analyses or models. Additionally, there are other techniques such as using machine learning models to impute missing categorical values, which can be more sophisticated and might provide better results in some cases.

Ultimately, the decision on how to handle missing categorical values should be made based on a combination of factors, including dataset size, the importance of the categorical feature, and the specific goals of your analysis or modeling.

Filling missing values in numerical columns requires different approaches, and the choice of method depends on the nature of the data and the specific context of your analysis. Here are some common strategies for handling missing numerical values:

1. **Mean or Median Imputation**:
   - **Pros**:
     - Simple and easy to implement.
     - Preserves the overall central tendency of the data.
   - **Cons**:
     - May not be suitable if the data has outliers, as it can be sensitive to extreme values.
     - Can lead to underestimation or overestimation of variability in the data.

   - **When to Use**: Mean or median imputation is a reasonable choice when missing values are missing completely at random and the data does not have severe outliers.

2. **Regression Imputation**:
   - **Pros**:
     - Provides a more accurate estimate of missing values by considering relationships with other variables.
   - **Cons**:
     - Requires more complex modeling, which can be computationally expensive.
     - Assumes a linear relationship between variables, which may not always hold.

   - **When to Use**: Regression imputation is useful when there are strong relationships between the missing variable and other variables in the dataset. It can provide more accurate imputations than simple mean or median imputation.

3. **K-Nearest Neighbors (K-NN) Imputation**:
   - **Pros**:
     - Takes into account the similarity between data points, making it robust to outliers and non-linear relationships.
   - **Cons**:
     - Computationally intensive, especially for large datasets.
     - The choice of the number of neighbors (k) can impact results.

   - **When to Use**: K-NN imputation is useful when the data points are not independent, and imputed values should be close to the values of neighboring data points.

4. **Multiple Imputation**:
   - **Pros**:
     - Provides a robust way to handle missing data by generating multiple imputed datasets and combining results.
     - Accounts for uncertainty in imputed values.
   - **Cons**:
     - Can be computationally intensive and complex to implement.

   - **When to Use**: Multiple imputation is suitable when you want to account for uncertainty and obtain more accurate imputations. It's particularly useful when missing data are not missing completely at random.

5. **Domain-Specific Imputation**:
   - **Pros**:
     - Customized imputation methods based on domain knowledge may yield the most accurate results.
   - **Cons**:
     - Requires expertise in the specific domain.

   - **When to Use**: Domain-specific imputation methods are valuable when you have a deep understanding of the data and can design imputation strategies tailored to the dataset's characteristics.

The choice of imputation method should be made carefully, considering the nature of the data, the presence of outliers, the amount of missing data, and the goals of your analysis. It's often a good practice to explore the data and assess the impact of different imputation methods on your results before making a final decision.

In [39]:
# 5. Write a function or functions to substitute the appropriate replacement value for each missing value
def fill_missing():
    df[['host_name','name']] = df[['host_name','name']].fillna('Anonymous')
    df['last_review'] = df['last_review'].fillna(pd.to_datetime('January 1, 1970'))
    df['reviews_per_month'] = df['reviews_per_month'].fillna(0)

fill_missing()

In [52]:
# Invalid values
# Continuous columns should be integers instead of floats except latitude and longitude
print(df[continuous+excluded].agg(['min','max']).unstack())
print('Count of invalid values',{col:pd.to_numeric(df[col], errors='coerce').isna().sum() 
                                 for col in continuous+excluded})
print({category: df[category].nunique() for category in categorical})

minimum_nights                  min    1.000000e+00
                                max    1.250000e+03
number_of_reviews               min    0.000000e+00
                                max    6.290000e+02
reviews_per_month               min    0.000000e+00
                                max    5.850000e+01
calculated_host_listings_count  min    1.000000e+00
                                max    3.270000e+02
latitude                        min    4.049979e+01
                                max    4.091306e+01
longitude                       min   -7.424442e+01
                                max   -7.371299e+01
price                           min    0.000000e+00
                                max    1.000000e+04
id                              min    2.539000e+03
                                max    3.648724e+07
dtype: float64
Count of invalid values {'minimum_nights': 0, 'number_of_reviews': 0, 'reviews_per_month': 0, 'calculated_host_listings_count': 0, 'latitude': 0, 'longit

In [54]:
# 8. Write functions that identify and deal with invalid values
def fix_invalid():
    cats = ['minimum_nights','number_of_reviews','reviews_per_month','calculated_host_listings_count']
    df[cats] = df[cats].astype('int')

fix_invalid()

In [55]:
# 10. Save the DataFrame to a pickle file
df.to_pickle(pickled_output_dataframe)